### Profiling and Benchmarking  
1. 使用python进行简单的端到端基准测试，测量前向和后向传递的时间  
2. 使用NVIDIA Nsight System进行性能分析
3. 分析显存使用

#### 1. 端到端性能评估  
测量代码从输入到输出完成所需的总时长。大模型训练成本高昂,推理速度直接影响用户体验

- 推理场景的关键指标：
  - 首字延迟（TTFT）：用户输入后，看到模型输出第一个字需要的时间
  - 逐字延迟（TPOT）：生成过程中，每个字之间的时间间隔
  - 吞吐量：系统在单位时间内总共生成多少TOKEN
- 训练场景关键指标:
  - 每秒浮点运算数:硬件每秒实际跑了多少次
  - 模型算力利用率:模型实际跑出的算力占硬件峰值算例的比例

#### Slurn平台上的Submitt

In [ ]:
import submitit
import time

def add(a,b):
  time.sleep(10)
  return a+b

#AutoExecutor 会自动检测到在slurm环境下
executor = submitit.LocalExecutor(folder="slurm_logs")#这里未创建
#设置要求
executor.update_parameters(
  timeout_min = 5,
  cpus_per_task = 1,
  mem_gb = 1
)
print("提交任务")

job = executor.submit(add,5,7)
print("等待结果")
result = job.result()
print(result)

当需要进行大量重复实验时,submitit的核心优势是控制器与工作任务解耦  
- 控制器:打包函数和参数,生成脚本,通过sbatch命令发送给Slurm.job_result()只是一个可选的等待和接受的动作
- 工作任务:Slurm受到请求后,会在某个计算节点上启动一个全新的,独立的进程

#### 2. 前向和反向性能评估
- 前向传播:
  - 任务:数据进入模型,逐层计算激活值,最后得到损失值
  - 计算量:线性Transformer,计算量为2*参数量*Tokens
- 反向传播:
  - 任务:利用链式法则,从loss开始往回推,计算每一层权重和输入的梯度
  - 计算量:前向传播的两倍
- 完整的一次训练:6*参数量*TOKEN

1) 预热: GPU 在第一次运行代码时需要编译内核、初始化库和加载缓存。如果你直接测第一圈的时间，数据会非常难看。专家建议：先“空跑” 10 次，待运行稳定后再开始计时  
2) 必须进行 CUDA 同步： 这是最容易犯的错。CPU 和 GPU 是异步工作的，CPU 发完指令就跑下一行了，不管 GPU 算完没算完  
3) 核心指标:$MFU=\frac{模型实际有效运算量(FLOPs)}{硬件理论分值算力*实际耗时}$
 

#### 3. 混合精度训练
混合精度训练引入16位浮点数(FP16或BF16)
- 减少显存占用:可以用同样的显存训练更大的模型,或者使用更大的批量
- 加快训练速度:低精度计算可以充分利用硬件性能
但是混合精度训练也有一些挑战:  
- 数据溢出:
  - 上溢: 一个数因为太大,超出了当前数据类型能表示的最大范围,结果变成了无穷大
  - 下溢: 一个数因为太小,超出了当前数据类型表示的最小精度范围,结果变成了0 
- 舍入误差:当网络模型的反向梯度很小,一些FP32能够表示的数值可能不能满足FP16精度下的表示范围
